# **Applied Machine Learning Homework 5**
**Due 12 Dec,2022 (Monday) 11:59PM EST**

### Natural Language Processing
We will train a supervised model to predict if a movie has a positive or a negative review.

####  **Dataset loading & dev/test splits**

**1.0) Load the movie reviews dataset from NLTK library**

In [3]:
import nltk
nltk.download("movie_reviews")
from nltk.corpus import movie_reviews
import pandas as pd
from nltk.corpus import twitter_samples 
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
stop = stopwords.words('english')
import string
import re
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

[nltk_data] Downloading package movie_reviews to
[nltk_data]     C:\Users\sohnh\AppData\Roaming\nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sohnh\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\sohnh\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
negative_fileids = movie_reviews.fileids('neg')
positive_fileids = movie_reviews.fileids('pos')

pos_document = [(' '.join(movie_reviews.words(file_id)),category) for file_id in movie_reviews.fileids() for category in movie_reviews.categories(file_id) if category == 'pos']
neg_document = [(' '.join(movie_reviews.words(file_id)),category) for file_id in movie_reviews.fileids() for category in movie_reviews.categories(file_id) if category == 'neg']

# List of postive and negative reviews
pos_list = [pos[0] for pos in pos_document]
neg_list = [neg[0] for neg in neg_document]

**1.1) Make a data frame that has reviews and its label**

In [18]:
# code here
df_pos = pd.DataFrame(pos_list, columns = ['review'])
df_pos['label'] = 1

df_neg = pd.DataFrame(neg_list, columns = ['review'])
df_neg['label'] = 0

df = pd.concat([df_pos, df_neg]).reset_index(drop=True)
df

,review,label
0,films adapted from comic books have had plenty...,1
1,every now and then a movie comes along from a ...,1
2,you ' ve got mail works alot better than it de...,1
3,""" jaws "" is a rare film that grabs your attent...",1
4,moviemaking is a lot like being the general ma...,1
...,...,...
1995,"if anything , "" stigmata "" should be taken as ...",0
1996,"john boorman ' s "" zardoz "" is a goofy cinemat...",0
1997,the kids in the hall are an acquired taste . i...,0
1998,there was a time when john carpenter was a gre...,0


**1.2 look at the class distribution of the movie reviews**

In [19]:
# code here
df.groupby('label').size() # we have 1000 reviews from both cases, it is a balanced dataset

label
0    1000
1    1000
dtype: int64

**1.3) Create a development & test split (80/20 ratio):**

In [26]:
# code here
x = df['review'].values
y = df['label'].values

x_dev, x_test, y_dev, y_test = train_test_split(x, y, test_size = 0.2, random_state = 42)

#### **Data preprocessing**
We will do some data preprocessing before we tokenize the data. We will remove `#` symbol, hyperlinks, stop words & punctuations from the data. You may use `re` package for this. 

**1.4) Replace the `#` symbol with '' in every review**

In [33]:
# code here
x_dev = [review.replace('#', '') for review in x_dev]
x_test = [review.replace('#', '') for review in x_test]

**1.5) Replace hyperlinks with '' in every review**

In [29]:
# code here

# there was announcement saying we can ignore this question

**1.6) Remove all stop words**

In [41]:
# code here
x_dev = [' '.join([w for w in word_tokenize(review) if not w.lower() in stop]) for review in x_dev]
x_test = [' '.join([w for w in word_tokenize(review) if not w.lower() in stop]) for review in x_test]

**1.7) Remove all punctuations**

In [51]:
# code here
x_dev = [' '.join(word.strip(string.punctuation) for word in review.split()) for review in x_dev]
x_test = [' '.join(word.strip(string.punctuation) for word in review.split()) for review in x_test]

**1.8) Apply stemming on the development & test datasets using Porter algorithm**

In [58]:
#code here
ps = PorterStemmer()

x_dev = [' '.join([ps.stem(word) for word in word_tokenize(review)]) for review in x_dev]
x_test = [' '.join([ps.stem(word) for word in word_tokenize(review)]) for review in x_test]

#### **Model training**

**1.9) Create bag of words features for each review in the development dataset**

In [61]:
#code here
vector = CountVectorizer()

x_dev_bow = vector.fit_transform(x_dev)
x_test_bow = vector.transform(x_test)

**1.10) Train a Logistic Regression model on the development dataset**

In [68]:
#code here
model_bow = LogisticRegression(max_iter=500).fit(x_dev_bow, y_dev)

**1.11) Create TF-IDF features for each review in the development dataset**

In [65]:
#code here
vector2 = TfidfVectorizer()

x_dev_ti = vector2.fit_transform(x_dev)
x_test_ti = vector2.transform(x_test)

**1.12) Train the Logistic Regression model on the development dataset with TF-IDF features**

In [69]:
#code here
model_tfidf = LogisticRegression().fit(x_dev_ti, y_dev)

**1.13) Compare the performance of the two models on the test dataset. Explain the difference in results obtained?**

In [73]:
#code here
print("Accuracy from Bag of Words: ", model_bow.score(x_test_bow, y_test).round(3))
print("Accuracy from TfIdf: ", model_tfidf.score(x_test_ti, y_test).round(3))

# Accuracy results are similar, but the bag of words model performs slightly better

Accuracy from Bag of Words:  0.822
Accuracy from TfIdf:  0.81
